# Real-Time EEG Signal Processing with BrainFlow

Welcome! This notebook provides an in-depth explanation of how to process EEG data in real-time using the BrainFlow library. We'll walk through a Python script that:

- Initializes a synthetic EEG data stream using BrainFlow.
- Implements real-time filtering (bandpass and notch).
- Applies z-score normalization.
- Ensures each data point is filtered only once.
- Retrieves the most recent 1.5 seconds of processed EEG data, updating at least 10 times per second.

We'll cover the reasoning behind each step, especially those marked with `(why?)` in the code.

---

## Table of Contents

1. [Introduction](#introduction)
2. [Imports and Initialization](#imports-and-initialization)
3. [Understanding the EEGProcessor Class](#understanding-the-eegprocessor-class)
   - [BrainFlow Initialization](#brainflow-initialization)
   - [Sampling Rate and Window Sizes](#sampling-rate-and-window-sizes)
   - [EEG Channels](#eeg-channels)
   - [Data Buffers](#data-buffers)
4. [Real-Time Data Processing](#real-time-data-processing)
   - [Fetching New Data](#fetching-new-data)
   - [Channel-wise Processing](#channel-wise-processing)
     - [Bandpass Filtering](#bandpass-filtering)
     - [Notch Filtering](#notch-filtering)
     - [Z-Score Normalization](#z-score-normalization)
   - [Buffer Management](#buffer-management)
5. [Main Loop Execution](#main-loop-execution)
   - [Initial Sleep](#initial-sleep)
   - [Processing Rate](#processing-rate)
6. [Why We Do Certain Things](#why-we-do-certain-things)
   - [Raw Window Size of 10 Seconds](#raw-window-size-of-10-seconds)
   - [Processing Each Channel Separately](#processing-each-channel-separately)
   - [Using the Whole Buffer for Filtering](#using-the-whole-buffer-for-filtering)
   - [Initial Sleep of 2 Seconds](#initial-sleep-of-2-seconds)
   - [Processing Data 10 Times per Second](#processing-data-10-times-per-second)
7. [Running the Code](#running-the-code)
8. [Conclusion](#conclusion)

---

## Introduction

Electroencephalography (EEG) signals are inherently noisy and require careful preprocessing to extract meaningful information. Real-time processing adds an additional layer of complexity due to the need for efficiency and minimal latency.

In this notebook, we'll explore how to process EEG data in real-time, focusing on:

- **Filtering**: Removing unwanted frequencies using bandpass and notch filters.
- **Normalization**: Standardizing the data through z-score normalization.
- **Buffer Management**: Ensuring data points are filtered only once and managing data windows.
- **Performance**: Processing data at a rate suitable for real-time applications.

---

## Imports and Initialization

In [ ]:
import time
import numpy as np
from brainflow.board_shim import BoardShim, BrainFlowInputParams, BoardIds
from scipy.signal import butter, lfilter, iirnotch

- **time**: Used for timing events and controlling the loop rate.
- **numpy**: Provides efficient array operations.
- **brainflow**: A library for interfacing with various EEG devices.
- **scipy.signal**: Contains signal processing functions like filters.

---

## Understanding the EEGProcessor Class

Let's dissect the `EEGProcessor` class step by step.

### BrainFlow Initialization

In [ ]:
class EEGProcessor:
    def __init__(self):
        # Initialize BrainFlow
        BoardShim.enable_dev_board_logger()
        params = BrainFlowInputParams()
        self.board_id = BoardIds.SYNTHETIC_BOARD.value
        self.board = BoardShim(self.board_id, params)
        self.board.prepare_session()
        self.board.start_stream()
        print("BrainFlow streaming started...")

- **BrainFlow Logger**: Enables logging for debugging purposes.
- **BrainFlowInputParams**: Holds connection parameters (empty here since we're using a synthetic board).
- **Synthetic Board**: Used for simulation; replace `BoardIds.SYNTHETIC_BOARD.value` with your actual board ID when using real hardware.
- **Session Preparation and Stream Start**: Initializes the board and starts data streaming.

### Sampling Rate and Window Sizes

In [ ]:
        # Sampling rate and window size
        self.sampling_rate = BoardShim.get_sampling_rate(self.board_id)
        self.window_size_sec = 1.5  # seconds
        self.window_size_samples = int(self.window_size_sec * self.sampling_rate)

        # We set raw window size to 10 seconds (why?)
        self.window_size_raw = int(10 * self.sampling_rate)
        self.lowcut = 1.0
        self.highcut = 50.0
        self.notch = 60.0


- **Sampling Rate**: The number of samples per second collected by the EEG device.
- **Processing Window Size**: 1.5 seconds, converted to samples.
- **Raw Window Size**: 10 seconds. **Why?** We'll explain this [later](#raw-window-size-of-10-seconds).
- **Filter Parameters**:
  - **Bandpass Filter**: Passes frequencies between 1 Hz and 50 Hz.
  - **Notch Filter**: Attenuates 60 Hz frequency (common powerline noise in some regions).

### EEG Channels

In [ ]:
        # Get EEG channels
        self.eeg_channels = BoardShim.get_eeg_channels(self.board_id)


- Retrieves the indices of EEG channels from the board.

### Data Buffers

In [ ]:
        # Initialize buffers
        self.raw_data_buffer = np.empty((len(self.eeg_channels), 0))
        self.processed_data_buffer = np.empty((len(self.eeg_channels), 0))


- **raw_data_buffer**: Stores unprocessed EEG data.
- **processed_data_buffer**: Stores filtered and normalized EEG data.
- Initialized as empty arrays with dimensions corresponding to the number of EEG channels.

---

## Real-Time Data Processing

### Fetching New Data


In [ ]:
    def get_recent_data(self):
        data = self.board.get_board_data()
        if data.shape[1] == 0:
            # No new data
            pass
        else:
            # Append new raw data to the raw_data_buffer
            eeg_data = data[self.eeg_channels, :]
            self.raw_data_buffer = np.hstack((self.raw_data_buffer, eeg_data))
            # Process new data
            # ...

- **get_board_data()**: Fetches new data from the board and clears the internal buffer.
- **eeg_data**: Extracts EEG channels from the fetched data.
- **Appending to Buffer**: New data is added to the raw data buffer.

### Channel-wise Processing

#### Bandpass Filtering

In [ ]:
            for i in range(len(self.eeg_channels)):
                # It's important to process each channel separately (why?)
                # Get the channel data
                channel_data = self.raw_data_buffer[i, :].copy()

                # It's important to use the whole buffer for filtering (why?)
                # Bandpass filter
                b, a = butter(2, [self.lowcut, self.highcut], btype='band', fs=self.sampling_rate)
                channel_data = lfilter(b, a, channel_data)


- **Processing Each Channel Separately**: We'll explain [why this is important](#processing-each-channel-separately).
- **Using the Whole Buffer**: Filtering requires past data; we'll discuss [why we use the whole buffer](#using-the-whole-buffer-for-filtering).
- **Butterworth Bandpass Filter**:
  - **Order**: 2 (controls the steepness of the filter).
  - **Cutoff Frequencies**: 1 Hz (lowcut) and 50 Hz (highcut).
  - **b, a**: Filter coefficients.
  - **lfilter**: Applies the filter to the data.

#### Notch Filtering


In [ ]:
                # Notch filter
                b, a = iirnotch(self.notch, 30, fs=self.sampling_rate)
                channel_data = lfilter(b, a, channel_data)


- **Notch Filter**:
  - **Center Frequency**: 60 Hz (common powerline interference in North America).
  - **Quality Factor**: 30 (determines the bandwidth of the notch).
  - **iirnotch**: Generates the notch filter coefficients.
  - **lfilter**: Applies the notch filter to the data.

#### Z-Score Normalization


In [ ]:
                # Z-score
                mean = np.mean(channel_data)
                std = np.std(channel_data)
                if std == 0:
                    std = 1  # Prevent division by zero
                channel_data = (channel_data - mean) / std
                new_processed_data[i, :] = channel_data

- **Mean and Standard Deviation**: Calculated for the channel data.
- **Normalization**: Subtracts the mean and divides by the standard deviation.
- **Handling Zero Standard Deviation**: Prevents division by zero by setting std to 1 if it's zero.

### Buffer Management


In [ ]:
            self.processed_data_buffer = np.hstack((self.processed_data_buffer, new_processed_data))

            # Trim buffers to maintain only the last window_size_samples
            max_buffer_size = self.window_size_samples * 2  # Keep some extra data
            if self.raw_data_buffer.shape[1] > self.window_size_raw:
                self.raw_data_buffer = self.raw_data_buffer[:, -self.window_size_raw:]
            if self.processed_data_buffer.shape[1] > max_buffer_size:
                self.processed_data_buffer = self.processed_data_buffer[:, -max_buffer_size:]

        if self.processed_data_buffer.shape[1] >= self.window_size_samples:
            recent_data = self.processed_data_buffer[:, -self.window_size_samples:]
        else:
            recent_data = self.processed_data_buffer

        return recent_data


- **Appending Processed Data**: Adds the newly processed data to the processed data buffer.
- **Trimming Buffers**:
  - **raw_data_buffer**: Trimmed to the last 10 seconds of data.
  - **processed_data_buffer**: Trimmed to twice the window size (3 seconds) to have some extra data if needed.
- **Returning Recent Data**:
  - If enough data is available, returns the most recent 1.5 seconds.
  - Otherwise, returns all available processed data.

---

## Main Loop Execution


In [ ]:
def main():
    eeg_processor = EEGProcessor()
    time.sleep(2)  # Wait until we can fill the buffer (why?)

    try:
        while True:
            start_time = time.time()
            recent_data = eeg_processor.get_recent_data()
            # Do something with recent_data
            print("Recent data shape:", recent_data.shape)
            # We want to process data 10 times a second (why?)
            elapsed_time = time.time() - start_time
            sleep_time = max(0, 0.1 - elapsed_time)
            time.sleep(sleep_time)
    except KeyboardInterrupt:
        print("Stopping...")
    finally:
        eeg_processor.stop()

if __name__ == "__main__":
    main()


- **Initial Sleep**: Waits for 2 seconds before starting processing. We'll explain [why this is necessary](#initial-sleep-of-2-seconds).
- **Processing Loop**:
  - Calls `get_recent_data()` to retrieve the latest processed data.
  - Prints the shape of the recent data (placeholder for actual processing).
- **Processing Rate**: Adjusted to process data 10 times per second. We'll discuss [why this rate is chosen](#processing-data-10-times-per-second).
- **Graceful Exit**: Handles keyboard interruption and ensures resources are released.

---

## Why We Do Certain Things

Let's address the `(why?)` comments in the code and explain the reasoning behind each decision.

### Raw Window Size of 10 Seconds

**Why do we set the raw window size to 10 seconds?**


```python
# We set raw window size to 10 seconds (why?)
self.window_size_raw = int(10 * self.sampling_rate)
```

**Explanation**:

- **Filtering Requires Past Data**:
  - Filters, especially IIR filters like Butterworth, have memory; they rely on previous input values.
- **Warm-up Period**:
  - When applying filters, the initial outputs may not be reliable due to lack of prior data.
- **Sufficient Data for Stable Filtering**:
  - Having a larger buffer (10 seconds) ensures that the filter has enough past data to produce stable and accurate filtered signals.
- **Balance Between Memory and Performance**:
  - 10 seconds is a reasonable compromise to retain sufficient data without excessive memory usage.

### Processing Each Channel Separately

**Why is it important to process each channel separately?**

```python
# It's important to process each channel separately (why?)
for i in range(len(self.eeg_channels)):
    # Process channel i
```

**Explanation**:

- **Independent Signals**:
  - EEG channels represent different electrodes placed on the scalp, each measuring local electrical activity.
- **Unique Signal Characteristics**:
  - Each channel may have different noise levels, artifacts, and signal characteristics.
- **Avoiding Cross-Channel Contamination**:
  - Processing channels separately prevents mixing signals, which could corrupt the data.
- **Channel-Specific Filters**:
  - Allows for individual adjustments if certain channels require different processing in the future.

### Using the Whole Buffer for Filtering

**Why is it important to use the whole buffer for filtering?**

```python
# It's important to use the whole buffer for filtering (why?)
channel_data = self.raw_data_buffer[i, :].copy()
```

**Explanation**:

- **Filter Statefulness**:
  - Filters like the IIR Butterworth filter are stateful, meaning their output depends on previous inputs.
- **Continuous Filtering**:
  - By applying the filter to the entire buffer, we maintain continuity in the filtered signal.
- **Avoiding Edge Effects**:
  - Filtering only the new data would introduce artifacts at the boundaries due to abrupt changes.
- **Consistency**:
  - Ensures that each data point is filtered only once and maintains temporal consistency in the signal.

### Initial Sleep of 2 Seconds

**Why do we wait for 2 seconds before starting the processing loop?**

```python
time.sleep(2)  # Wait until we can fill the buffer (why?)
```

**Explanation**:

- **Buffer Warm-up**:
  - Allows the raw data buffer to accumulate enough data (at least 2 seconds worth).
- **Stable Filtering**:
  - Ensures that when we start filtering, the buffer has sufficient past data for the filters to operate correctly.
- **Preventing Initial Artifacts**:
  - Starting processing immediately might result in unreliable outputs due to insufficient data for the filters.

### Processing Data 10 Times per Second

**Why do we want to process data 10 times a second?**

```python
# We want to process data 10 times a second (why?)
sleep_time = max(0, 0.1 - elapsed_time)
time.sleep(sleep_time)
```

**Explanation**:

- **Real-Time Responsiveness**:
  - Updating 10 times per second (every 100 ms) provides a good balance between responsiveness and computational load.
- **Human Perception**:
  - Visual or feedback systems updating at 10 Hz appear smooth to human observers.
- **Data Overlap**:
  - With a window size of 1.5 seconds and updates every 0.1 seconds, there's significant overlap between consecutive data windows.
- **Sufficient for Many Applications**:
  - This rate is suitable for many real-time EEG applications like neurofeedback, where immediate processing is essential.

---

## Running the Code

To run the code:

1. **Install Required Libraries**:

   ```bash
   pip install brainflow numpy scipy
   ```

2. **Save the Code**:

   Save the script in a file, e.g., `real_time_eeg.py`.

3. **Run the Script**:

   ```bash
   python real_time_eeg.py
   ```

4. **Observe the Output**:

   The script will print the shape of the recent data array every 0.1 seconds.

5. **Stop the Script**:

   Press `Ctrl+C` to stop the script gracefully.

---

## Conclusion

In this notebook, we've explored how to process EEG data in real-time using Python and BrainFlow. We've covered:

- **Initialization**: Setting up the EEG data stream and buffers.
- **Filtering**: Applying bandpass and notch filters to clean the signal.
- **Normalization**: Standardizing the data using z-score normalization.
- **Buffer Management**: Maintaining and trimming data buffers to ensure efficiency.
- **Processing Loop**: Implementing a loop that processes data at a suitable rate for real-time applications.

Understanding the reasoning behind each step is crucial for developing effective EEG data processing pipelines, especially when dealing with real-time requirements.

---

**Next Steps**:

- **Integrate with Real EEG Devices**: Replace the synthetic board with actual EEG hardware.
- **Implement Advanced Processing**: Explore techniques like artifact removal, feature extraction, or machine learning models.
- **Visualize the Data**: Use libraries like Matplotlib or PyQtGraph to visualize the EEG signals in real-time.
- **Optimize Performance**: For higher data rates or more complex processing, consider optimizing the code or using multi-threading.

---

**References**:

- [BrainFlow Documentation](https://brainflow.readthedocs.io/en/stable/)
- [Scipy Signal Processing](https://docs.scipy.org/doc/scipy/reference/signal.html)
- [EEG Signal Processing Techniques](https://www.sciencedirect.com/topics/medicine-and-dentistry/eeg-signal-processing)

If you have any questions or need further clarification on any part of this notebook, feel free to ask!